### Set working directory

In [1]:
%cd ~/theta_methyl/

/Users/bartosz/theta_methyl


### Download FASTQ file

In [11]:
!wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR617/001/SRR6176401/SRR6176401.fastq.gz

--2023-03-20 22:11:01--  ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR617/001/SRR6176401/SRR6176401.fastq.gz
           => 'SRR6176401.fastq.gz'
Resolving ftp.sra.ebi.ac.uk... 193.62.193.138
Connecting to ftp.sra.ebi.ac.uk|193.62.193.138|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /vol1/fastq/SRR617/001/SRR6176401 ... done.
==> SIZE SRR6176401.fastq.gz ... 1081731478
==> PASV ... done.    ==> RETR SRR6176401.fastq.gz ... done.
Length: 1081731478 (1.0G) (unauthoritative)

SRR6176401.fastq.gz 100%[===================>]   1.01G  2.28MB/s    in 7m 13s  

2023-03-20 22:18:15 (2.38 MB/s) - 'SRR6176401.fastq.gz' saved [1081731478]



In [6]:
!head -4 <(gzcat SRR6176401.fastq.gz)

@SRR6176401.1 1/1
TTTTATATTTTTATTTTTTATAAATTATAATTTATTTTTATTGTATTTATGGTATTGGATTAATAATATTAGTGTAGCTAAATTAGTTAAAAGATTAATG
+
BBBFFFFFFFFFFIIIIIIIIIIIIIIIIIIIIIIIIFIIIIIFIIIIIIIIFFFIIIIIIIIIFFIIIIIIFIIFF07BFIIIFFFFFFFFFFFFFFFB


In [10]:
!conda info --envs

# conda environments:
#
base                  *  /Users/bartosz/anaconda3
MachineLearningCourse     /Users/bartosz/anaconda3/envs/MachineLearningCourse
filip                    /Users/bartosz/anaconda3/envs/filip
methyl_theta             /Users/bartosz/anaconda3/envs/methyl_theta
mothur                   /Users/bartosz/anaconda3/envs/mothur
ngs2                     /Users/bartosz/anaconda3/envs/ngs2
qiime2-2020.8            /Users/bartosz/anaconda3/envs/qiime2-2020.8
r4-base                  /Users/bartosz/anaconda3/envs/r4-base



In [12]:
!fastp 

fastp: an ultra-fast all-in-one FASTQ preprocessor
version 0.12.4
usage: fastp --in1=string [options] ... 
options:
  -i, --in1                            read1 input file name (string)
  -o, --out1                           read1 output file name (string [=])
  -I, --in2                            read2 input file name (string [=])
  -O, --out2                           read2 output file name (string [=])
  -6, --phred64                        indicates the input is using phred64 scoring (it'll be converted to phred33, so the output will still be phred33)
  -z, --compression                    compression level for gzip output (1 ~ 9). 1 is fastest, 9 is smallest, default is 2. (int [=2])
  -A, --disable_adapter_trimming       adapter trimming is enabled by default. If this option is specified, adapter trimming is disabled
  -a, --adapter_sequence               the adapter for read1. For SE data, if not specified, the adapter will be auto-detected. For PE data, this is used if R1/R2 a

In [11]:
!open fastp.html

### Download reference genome (let's download only one chromosome to make the analysis faster)
###### https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-56/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz

In [31]:
!wget https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-56/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz

--2023-03-21 20:53:34--  https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-56/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz
Resolving ftp.ensemblgenomes.ebi.ac.uk... 193.62.193.141
Connecting to ftp.ensemblgenomes.ebi.ac.uk|193.62.193.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9211107 (8.8M) [application/x-gzip]
Saving to: 'Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz'

Arabidopsis_thalian 100%[===================>]   8.78M  1.24MB/s    in 7.2s    

2023-03-21 20:53:41 (1.23 MB/s) - 'Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz' saved [9211107/9211107]



In [53]:
!head <(gzcat Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz)

>1 dna:chromosome chromosome:TAIR10:1:1:30427671:1 REF
CCCTAAACCCTAAACCCTAAACCCTAAACCTCTGAATCCTTAATCCCTAAATCCCTAAAT
CTTTAAATCCTACATCCATGAATCCCTAAATACCTAATTCCCTAAACCCGAAACCGGTTT
CTCTGGTTGAAAATCATTGTGTATATAATGATAATTTTATCGTTTTTATGTAATTGCTTA
TTGTTGTGTGTAGATTTTTTAAAAATATCATTTGAGGTCAATACAAATCCTATTTCTTGT
GGTTTTCTTTCCTTCACTTAGCTATGGATGGTTTATCTTCATTTGTTATATTGGATACAA
GCTTTGCTACGATCTACATTTGGGAATGTGAGTCTCTTATTGTAACCTTAGGGTTGGTTT
ATCTCAAGAATCTTATTAATTGTTTGGACTGTTTATGTTTGGACATTTATTGTCATTCTT
ACTCCTTTGTGGAAATGTTTGTTCTATCAATTTATCTTTTGTGGGAAAATTATTTAGTTG
TAGGGATGAAGTCTTTCTTCGTTGTTGTTACGCTTGTCATCTCATCTCTCAATGATATGG


##### Imagine this file contains the whole reference genome. How to replace the header of FASTA file by presenting only:
##### >1
##### ...
##### >2
##### ...
##### etc.? Replace the header by short chromosome name

In [4]:
!pwd

/Users/bartosz/theta_methyl


In [3]:
!zcat genome/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz | awk '/^>/{print ">" ++i; next}{print}'

zcat: can't stat: genome/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz (genome/Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz.Z): No such file or directory


In [5]:
!mv Arabidopsis_thaliana.TAIR10.dna.chromosome.1.fa.gz genome/

In [15]:
!bismark_genome_preparation --bowtie2 genome

Writing bisulfite genomes out into a single MFA (multi FastA) file

Bisulfite Genome Indexer version v0.24.0 (last modified: 19 May 2022)

A directory called /Users/bartosz/theta_methyl/genome/Bisulfite_Genome/ already exists. Already existing converted sequences and/or already existing Bowtie 2, HISAT2 or Minimap2) indices will be overwritten!


Step I - Prepare genome folders - completed



Total number of conversions performed:
C->T:	5435374
G->A:	5421151

Step II - Genome bisulfite conversions - completed


Bismark Genome Preparation - Step III: Launching the Bowtie 2 indexer
Please be aware that this process can - depending on genome size - take several hours!
Settings:
  Output files: "BS_CT.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
 

  Time to join reference sequences: 00:00:01
  Time to reverse reference sequence: 00:00:00
bmax according to bmaxDivN setting: 7565828
Using parameters --bmax 5674371 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 5674371 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:01
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:01
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:00
  Sanity-checking and returning
Building samples
Reserving space for 12 sample suffixes
Generating random suffixes
QSorting 12 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 12 samples
  (Using difference cover)
  Multikey QSorting samples time: 00:00:00
Calculating bucket sizes
Splittin

In [16]:
!fastp --in1 SRR6176401.fastq.gz --out1 SRR6176401.filtered.fastq.gz -t 5

Detecting adapter for single end input...
Detected adapter: TTATATTATGAAAGTTTTGAGAAGTA

Read1 before filtering:
total reads: 18885258
total bases: 1888525800
Q20 bases: 1832907690(97.0549%)
Q30 bases: 1741003315(92.1885%)

Read1 after filtering:
total reads: 18804298
total bases: 1768245951
Q20 bases: 1721381346(97.3497%)
Q30 bases: 1640652789(92.7842%)

Filtering result:
reads passed filter: 18804298
reads failed due to low quality: 22823
reads failed due to too many N: 6683
reads failed due to too short: 51454
reads with adapter trimmed: 936285
bases trimmed due to adapters: 23013760

JSON report: fastp.json
HTML report: fastp.html

fastp --in1 SRR6176401.fastq.gz --out1 SRR6176401.filtered.fastq.gz -t 5 
fastp v0.12.4, time used: 188 seconds


In [17]:
!open fastp.html

In [1]:
!bismark --bowtie2 genome/ SRR6176401.filtered.fastq.gz

Bowtie 2 seems to be working fine (tested command 'bowtie2 --version' [2.4.2])
Output format is BAM (default)
Alignments will be written out in BAM format. Samtools found here: '/Users/bartosz/anaconda3/bin/samtools'
Reference genome folder provided is genome/	(absolute path is '/Users/bartosz/theta_methyl/genome/)'
FastQ format assumed (by default)

Input files to be analysed (in current folder '/Users/bartosz/theta_methyl'):
SRR6176401.filtered.fastq.gz
Library is assumed to be strand-specific (directional), alignments to strands complementary to the original top or bottom strands will be ignored (i.e. not performed!)
Setting parallelization to single-threaded (default)

Summary of all aligner options:	-q --score-min L,0,-0.2 --ignore-quals
Current working directory is: /Users/bartosz/theta_methyl

Now reading in and storing sequence information of the genome specified in: /Users/bartosz/theta_methyl/genome/

chr 1 (30427671 bp)

Single-core mode: setting pid to 1

Single-end alignme

In [5]:
!conda activate py2


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


